# Chapter6. 다양한 데이터 타입 다루기

## 6.1 API는 어디서 찾을까

**분석에 사용할 DataFrame 생성**

In [0]:
file_location = "/FileStore/tables/data/retail-data/by-day/2010_12_01.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(file_location)

df.printSchema()
df.createOrReplaceTempView("dfTable")
df.show()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|2010-12-01 08:26:00| 2.75| 17850.0|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 22752|SET 7 BABUSHKA NE...| 2|2010-12-01 08:26:00| 7.65| 17850.0|United Kingdom|
 536365| 21730|GLASS STAR FROSTE...| 6|2010-12-01 08:26:00| 4.25| 17850.0|United Kingdom|
 536366| 22633|HAND WARMER UNION...| 6|2010-12-01 08:28:00| 1.85| 17850.0|United Kingdom|
 536366| 22632|HAND WARMER RED P...| 6|2010-12-01 08:28:00| 1.85| 17850.0|United Kingdom|
 536367| 84879|ASSORTED COLOUR B...| 32|2010-12-01 08:34:00| 1.69| 13047.0|United Kingdom|
 536367| 22745|POPPY'S PLAYHOUSE...| 6|2010-12-01 08:34:00| 2.1| 13047.0|United Kingdom|
 536367| 22748|POPPY'S PLAYHOUSE...| 6|2010-12-01 08:34:00| 2.1| 13047.0|United Kingdom|
 536367| 22749|FELTCRAFT PRINCES...| 8|2010-12-01 08:34:00| 3.75| 13047.0|United Kingdom|
 536367| 22310|IVORY KNITTED MUG...| 6|2010-12-01 08:34:00| 1.65| 13047.0|United Kingdom|
 536367| 84969|BOX OF 6 ASSORTED...| 6|2010-12-01 08:34:00| 4.25| 13047.0|United Kingdom|
 536367| 22623|BOX OF VINTAGE JI...| 3|2010-12-01 08:34:00| 4.95| 13047.0|United Kingdom|
 536367| 22622|BOX OF VINTAGE AL...| 2|2010-12-01 08:34:00| 9.95| 13047.0|United Kingdom|
 536367| 21754|HOME BUILDING BLO...| 3|2010-12-01 08:34:00| 5.95| 13047.0|United Kingdom|
 536367| 21755|LOVE BUILDING BLO...| 3|2010-12-01 08:34:00| 5.95| 13047.0|United Kingdom|
 536367| 21777|RECIPE BOX WITH M...| 4|2010-12-01 08:34:00| 7.95| 13047.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 20 rows

## 6.2 스파크 데이터 타입으로 변환하기

데이터 타입 변환은 **lit 함수**를 사용한다. lit 함수는 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환한다.

In [0]:
from pyspark.sql.functions import lit

df.select(lit(5), lit("five"), lit(5.0))

## 6.3 불리언 데이터 타입 다루기

불리언은 모든 필터링 작업의 기반이므로 데이터 분석에 필수적. 

불리언 구문은 and, or, true, false로 구성된다. 

불리언 구문을 사용해 true 또는 false로 평가되는 논리 문법을 만든다. 논리 문법은 데이터 로우를 필터링할 때 필요조건의 일치와 불일치를 판별하는데 사용.

In [0]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, False)

**문자열 표현식에 조건절 명시 - "일치하지 않음" 표현**

In [0]:
df.where("InvoiceNo = 536365")\
  .show(5, False)

In [0]:
df.where("InvoiceNo <> 536365")\
  .show(5, False)

**and 메서드나 or 메서드 사용**

불리언 표현식을 사용하는 경우 항상 모든 표현식을 and 메서드로 묶어 차례대로 필터를 적용해야 한다. 

차례대로 필터를 적용해야 하는 이유 : 불리언 문을 차례대로 표현해도 스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환하고, 동시에 모든 필터를 처리한다. 원한다면 and 구문으로 조건문을 만들 수도 있지만 차례로 조건을 나열하면 이해하기 쉽고 읽기가 편해진다. 반면 or 구문을 사용할 때는 반드시 동일한 구문에 조건을 정의해야 한다.

In [0]:
from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >=1 
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

**불리언 컬럼을 사용해 DataFrame 필터링**

In [0]:
from pyspark.sql.functions import instr

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
  .where("isExpensive")\
  .select("unitPrice", "isExpensive").show(5)

필터를 반드시 표현식으로 정의할 필요는 없다. 별도의 작업 없이 컬럼명을 사용해 필터를 정의하는 것도 가능하다.

In [0]:
from pyspark.sql.functions import expr

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
  .where("isExpensive")\
  .select("Description", "UnitPrice").show(5)

## 6.4 수치형 데이터 타입 다루기 

카운트(count)는 빅데이터 처리에서 필터링 다음으로 많이 수행하는 작업이다. 대부분은 수치형 데이터 타입을 사용해 연산 방식을 정의하면 된다. 

* pow 함수 : 표시된 지수만큼 컬럼명 값을 거듭제곱한다.

In [0]:
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

소수점 자리를 없애기 위해 Integer 데이터 타입으로 형변환하기도 하지만 반올림을 주로 사용함. 
* round 함수 : 소수점 값이 정확히 중간값 이상이라면 반올림
* bround 함수 : 내림

In [0]:
from pyspark.sql.functions import lit, round, bround

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

두 컬럼 사이의 상관관계를 계산하는 것도 수치형 연산 작업 중 하나. 

DataFrame의 통계용 함수나 메서드를 사용해 피어슨 상관관계 계산 가능.

In [0]:
from pyspark.sql.functions import corr

df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

* describe 메서드 : 관련 컬럼에 대한 집계(count), 평균(mean), 표준편차(stddev), 최솟값(min), 최댓값(max)를 계산

단, 통계 스키마는 변경될 수 있으므로 describe 메서드는 콘솔 확인용으로만 사용.

In [0]:
df.describe().show()

정확한 수치가 필요하면 함수를 임포트하고 해당 컬럼에 적용하는 방식으로 직접 집계 수행 가능.

In [0]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

**StatFunctions 패키지** : 다양한 통계함수 제공 (stat 속성을 사용해 접근)

* approxQuantile 메서드 : 데이터의 백분위수를 정확하게 계산하거나 근사치를 계산할 있음

In [0]:
olName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError)

* freqItems 메서드 : 교차표나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드

단, 연산 결과가 너무 크면 화면에 모두 보이지 않을 수 있음.

In [0]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

* monotonically_increasing_id 함수 : 모든 로우에 고유 ID 값을 추가. (0부터 시작)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(2)

## 6.5 문자열 데이터 타입 다루기

문자열을 다루는 작업은 거의 모든 데이터 처리 과정에서 발생. 로그 파일에 정규 표현식을 사용해 데이터 추출, 데이터 치환, 문자열 존재 여부, 대/소문자 변환 처리 등의 작업 가능

* initcap 함수 : 주어진 문자열에서 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경

In [0]:
from pyspark.sql.functions import initcap

df.select(initcap(col("Description"))).show(5)

* lower 함수 : 문자열 전체를 소문자로 변경
* upper 함수 : 문자열 전체를 대문자로 변경

In [0]:
from pyspark.sql.functions import lower, upper

df.select(col("Description"),
   lower(col("Description")),
   upper(lower(col("Description")))).show(2)

* lpad, ltrim, rpad, rtrim, trim 함수 : 문자열 주변의 공백을 제거하거나 추가하는 작업

In [0]:
from pyspark.sql.functions import lit, ltrim, rtrim, lpad, rpad, trim

df.select(
  ltrim(lit("     HELLO     ")).alias("ltrim"),
  rtrim(lit("     HELLO     ")).alias("rtrim"),
  trim(lit("     HELLO     ")).alias("trim"),
  lpad(lit("HELLO"), 3, " ").alias("lp"),
  rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

### 6.5.1 정규 표현식

**정규 표현식** 이란?

문자열의 존재 여부를 확인하거나 일치하는 모든 문자열을 치환할 때 사용. 정규 표현식을 사용해 문자열에서 값을 추출하거나 다른 값으로 치환하는데 필요한 규칙 모음 정의 가능.

스파크에서는 regexp_extract 함수와 regexp_replace 함수를 이용해 값을 추출하고 치환하는 역할을 수행한다.

In [0]:
# regexp_replace 함수를 사용해 description 컬럼의 값을 'COLOR'로 치환

from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GREEN|BLUE"

df.select(
  regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
  col("Description")).show(2)

* translate 함수 : 교체 문자열에서 색인된 문자에 해당하는 모든 문자를 치환.

아래의 예제에서는 L=1, E=3, T=7로 치환.

In [0]:
from pyspark.sql.functions import translate

df.select(translate(col("Description"), "LEET", "1337"), col("Description"))\
  .show(2)

In [0]:
# 처음 나타난 색상 이름을 추출하는 것과 같은 작업 수행 가능

from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"

df.select(
  regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
  col("Description")).show(2)

* contains 메서드 : 값 추출 없이 단순히 값의 존재 여부를 확인하고자 할 때, 인수로 입력된 값이 컬럼의 문자열에 존재하는지 불리언 타입으로 반환

단, 파이썬과 SQL에서는 **instr 함수**를 사용해 값의 존재 여부 확인

In [0]:
from pyspark.sql.functions import instr

containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1

df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
  .where("hasSimpleColor")\
  .select("Description").show(3, False)

동적으로 인수의 개수가 변하는 상황?

(Scalar)

* varargs : 값 목록을 인수로 변환해 함수에 전달할 때 사용하는 스칼라 고유 기능. 

varargs 기능을 사용하면 임의 길이의 배열을 효율적으로 다룰 수 있다. 

ex) select 메서드와 varargs를 함께 사용하면 원하는 만큼 동적으로 컬럼을 생성할 수 있다.


(Python) 


* locate 함수 : 문자열의 위치(1부터 시작)를 정수로 반환

그 다음 위치 정보를 불리언 타입으로 변환. 

ex) locate 함수를 확장해 입력 값의 최소공배수를 구하거나 소수 여부 판별 가능.

In [0]:
from pyspark.sql.functions import expr, locate

simpleColors = ["black", "white", "red", "green", "blue"]

def color_locator(column, color_string):
  return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string)

selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*"))

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
  .select("Description").show(3, False)

## 6.6 날짜와 타임스탬프 데이터 다루기 

* 스파크는 두 가지 종류의 시간 관련 정보만 관리하는데 하나는 **달력 형태의 날짜(date)**, 다른 하나는 날짜와 시간 정보를 모두 가지는 **타임스탬프(timestamp)**

* 스파크의 inferSchema 옵션이 활성화된 경우 날짜와 타임스탬프를 포함해 컬럼의 데이터 타입을 최대한 정확하게 식별하려 시도함. 
* 스파크는 특정 날짜 포맷을 명시하지 않아도 자체적으로 식별해 데이터를 읽을 수 있음.

*TimestampType 클래스는 초 단위의 정밀도까지만 지원하므로 밀리세컨드나 마이크로세컨드 단위를 다루기 위해서는 Long 데이터 타입으로 데이터를 변환해 처리. 그 이상의 정밀도는 TimestampType으로 변환될 때 제거된다.*

(스파크는 자바의 날짜와 타임스탬프를 사용해 표준체계를 따른다.)

In [0]:
# 오늘 날짜와 현재 타임스탬프 값 구하기

from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10)\
  .withColumn("today", current_date())\
  .withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

In [0]:
# 오늘을 기준으로 5일 전후의 날짜 구하기
# date_add 함수와 date_sub 함수는 컬럼과 더하거나 뺄 날짜의 수를 인수로 전달해야 함.

from pyspark.sql.functions import date_add, date_sub

dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

* datediff 함수 : 두 날짜 사이의 일 수를 반환
* months_between 함수 : 두 날짜 사이의 개월 수를 반환
* to_date 함수 : 문자열을 날짜로 변환할 수 있으며, 필요에 따라 날짜 포맷도 함께 지정 가능. 단, 날짜 포맷은 반드시 자바의 SimpleDateFormat 클래스가 지원하는 포맷 사용

In [0]:
from pyspark.sql.functions import datediff, months_between, to_date

dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
  .select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(
  to_date(lit("2016-01-01")).alias("start"),
  to_date(lit("2017-05-22")).alias("end"))\
  .select(months_between(col("start"), col("end"))).show(1)

In [0]:
from pyspark.sql.functions import to_date, lit

spark.range(5).withColumn("date", lit("2017-01-01"))\
  .select(to_date(col("date"))).show(1)

스파크는 날짜를 파싱할 수 없다면 에러 대신 null 값을 반환함. 따라서 다단계 처리 파이프라인에서는 조금 까다로울 수 있다. 데이터 포맷이 지정된 데이터에서 또 다른 포맷의 데이터가 나타날 수 있기 때문. 

ex) 년-월-일 형태가 아닌 년-일-월 형태의 날짜 포맷 사용시 날짜를 파싱할 수 없으므로 null 값을 반환함.

In [0]:
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1)

자바의 SimpleDateFormat 표준에 맞춰 날짜 포맷 지정

* to_date 함수 : 필요에 따라 날짜 포맷 지정
* to_timestamp 함수 : 반드시 날짜 포맷 지정

In [0]:
# to_date 예제

from pyspark.sql.functions import to_date

dateFormat = "yyyy-dd-MM"

cleanDateDF = spark.range(1).select(
  to_date(lit("2017-12-11"), dateFormat).alias("date"),
  to_date(lit("2017-20-12"), dateFormat).alias("date2"))

cleanDateDF.createOrReplaceTempView("dateTable2")

In [0]:
# to_timestamp 예제

from pyspark.sql.functions import to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

날짜를 비교할 때는 날짜나 타임스탬프 타입을 사용하거나 yyyy-MM-dd 포맷에 맞는 문자열 지정

In [0]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

## 6.7 null 값 다루기

DataFrame에서 빠져 있거나 비어 있는 데이터를 표현할 때는 null 값을 사용하는 것이 빈 문자열이나 대체 값을 사용하는 것보다 좋다. - 최적화를 수행할 수 있기 때문.

DataFrame의 하위 패키지인 .na를 사용하는 것이 DataFrame에서 null 값을 다루는 기본 방식. 

<null 값을 다루는 방법>
* null 값을 제거
* 전역 또는 컬럼 단위로 null 값을 특정 값으로 채워 넣는 방법

### 6.7.1 coalesce

* coalesce 함수 : 인수로 지정한 여러 컬럼 중 null이 아닌 첫 번째 값 반환 (모든 컬럼이 null이 아닌 값을 가진 경우 첫 번째 컬럼의 값 반환)

In [0]:
from pyspark.sql.functions import coalesce

df.select(coalesce(col("Description"), col("CustomerId"))).show()

### 6.7.2 ifnull, nullif, nvl, nvl2

coalesce 함수와 유사한 결과를 얻을 수 있는 SQL 함수
* ifnull 함수 : 첫 번째 값이 null이면 두 번째 값 반환 (첫 번째 값이 null이 아니면 첫 번째 값 반환)
* nullif 함수 : 두 값이 같으면 null 반환 (두 값이 다르면 첫 번째 값 반환)
* nvl 함수 : 첫 번째 값이 null이면 두 번째 값 반환 (첫 번째 값이 null 이 아니면 첫 번째 값 반환)
* nlv2 함수 : 첫 번째 값이 null이 아니면 두 번째 값 반환 (첫 번째 값이 null이면 세 번째 인수로 지정된 값 반환)

### 6.7.3 drop

drop 메소드 : null 값을 가진 로우 제거. (null 값을 가진 모든 로우 제거)


* 단, SQL을 사용한다면 컬럼별로 수행

-- SQL
SELECT * 
FROM dfTable 
WHERE Description IS NOT NULL

In [0]:
df.na.drop()

In [0]:
df.na.drop("any") # 인수로 any를 지정한 경우 로우의 컬럼 값 중 하나라도 null 값을 가지면 해당 로우 제거 

In [0]:
df.na.drop("all") # 인수를 all로 지정한 경우 모든 컬럼의 값이 null이거나 NaN인 경우에만 해당 로우 제거

In [0]:
# drop 메서드에 배열 형태의 컬럼을 인수로 적용 가능
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

### 6.7.4 fill

fill 함수 : 하나 이상의 컬럼을 특정 값으로 채우는 함수 - 채워 넣을 값과 컬럼 집합으로 구성된 맵을 인수로 사용

ex) String 데이터 타입의 컬럼에 존재하는 null 값을 다른 값으로 채우기

In [0]:
df.na.fill("All Null values become this string")

* df.na.fill(5:Integer) : Integer 데이터 타입의 컬럼에 존재하는 null 값을 다른 값으로 채워넣기
* df.na.fill(5:Double)  : Double 데이터 타입의 컬럼에 적용

In [0]:
df.na.fill("all", subset=["StockCode", "InvoiceNo"]) # 다수의 컬럼에 적용할 경우 적용하고자 하는 컬럼명을 배열로 만들어 인수로 사용

스칼라 Map 타입을 사용해 다수의 컬럼에 fill 메서드 적용 가능

* 키(key) : 컬럼명
* 값(value) : null 값을 채우는데 사용할 값

In [0]:
fill_cols_vals = {"StockCode" : 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

### 6.7.5 replace

replace 메소드 : 조건에 따라 다른 값으로 대체하는 것으로, 변경하고자 하는 값과 원래 값의 데이터 타입이 같아야함.

In [0]:
df.na.replace([""], ["UNKNOWN"], "Description")

## 6.8 정렬하기

DataFrame을 정렬할 때 다음과 같은 함수로 null 값이 표시되는 기준 지정 가능

* acc_nulls_first
* desc_nulls_fisrt
* acc_nulls_last
* desc_nulls_last

## 6.9 복합 데이터 타입 다루기

복합 데이터 타입을 사용하면 해결하려는 문제에 더욱 적합한 방식으로 데이터 구성 및 구조화 가능.

< 복합 데이터 타입의 종류 >
* 구조체(struct)
* 배열(array)
* 맵(map)

### 6.9.1 구조체

구조체 : DataFrame 내부의 DataFrame - 쿼리문에서 다수의 컬럼을 괄호로 묶어 구조체 생성 가능.

In [0]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

복합 데이터타입을 가진 DataFrame도 다른 DataFrame을 조회하는 것과 마찬가지로 사용이 가능하지만 점(.)을 사용하거나 getField 메소드를 사용해야 한다.

In [0]:
complexDF.select("complex.Description")

In [0]:
complexDF.select(col("complex").getField("Description"))

별표(*) 문자를 사용하면 모든 값을 조회하는 것도 가능.

In [0]:
complexDF.select("complex.*")

### 6.9.2 배열

배열을 정의하기 위해 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환과정 필요. 

**split**

split 함수에 구분자를 인수로 전달해 인수에 따라 배열로 변환.

In [0]:
from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

split 함수는 스파크에서 복합 데이터 타입을 다른 컬럼처럼 다룰 수 있는 기능을 가지기 때문에 파이썬과 유사한 문법으로 배열 값 조회 가능.

In [0]:
df.select(split(col("Description"), " ").alias("array_col"))\
  .selectExpr("array_col[0]").show(2)

**배열의 길이** - 배열의 크기 조회

In [0]:
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

**array_contains**

array_contains 함수를 사용해 배열에 특정 값이 존재하는지 여부 확인 가능 (true / false 반환)

In [0]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

**explode**

explode 함수는 배열 타입의 컬럼을 입력 받아 입력된 컬럼의 배열값에 포함된 모든 값을 로우로 변환. 나머지 컬럼 값은 중복되어 표시.

< 텍스트로 이루어진 explode 함수 처리 과정 >

ex) "Hello World"와 "other col"의 컬럼 값이 존재할 때 "Hello World"에 적용

* split 함수 적용 -> ["Hello", "World"], "other col"
* explode 함수 적용 
  -> "Hello", "other col"
     "World", "other col"

In [0]:
from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded").show(2)

### 6.9.3 맵

맵 : map 함수와 컬럼의 키-값 쌍을 이용해 생성하고, 배열과 동일한 방법으로 값 선택 가능.

In [0]:
from pyspark.sql.functions import create_map, col

df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .show(2)

+--------------------+
 complex_map|
+--------------------+
{WHITE HANGING HE...|
{WHITE METAL LANT...|
+--------------------+
only showing top 2 rows

적합한 키를 이용해 데이터 조회 가능 (해당 키가 존재하지 않으면 null 값 반환)

In [0]:
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

+--------------------------------+
complex_map[WHITE METAL LANTERN]|
+--------------------------------+
 null|
 536365|
+--------------------------------+
only showing top 2 rows

맵은 분해해 컬럼으로 변환 가능

In [0]:
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("explode(complex_map)").show(2)

+--------------------+------+
 key| value|
+--------------------+------+
WHITE HANGING HEA...|536365|
 WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows

## 6.10 JSON 다루기

스파크는 JSON 데이터를 다루기 위한 기능 지원
* 문자열 형태의 JSON을 직접 조회 가능
* JSON 파싱 혹은 JSON 객체 생성 가능

ex) JSON 컬럼 생성

In [0]:
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

* get_json_object 함수 : JSON 객체(딕셔너리 혹은 배열)를 인라인 쿼리로 조회 가능
* json_tuple : 중첩이 없는 단일 수준의 JSON 객체의 경우

In [0]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
  get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),
  json_tuple(col("jsonString"), "myJSONKey")).show(2)

+------+--------------------+
column| c0|
+------+--------------------+
 2|{"myJSONValue":[1...|
+------+--------------------+

* to_json 함수 : StructType을 JSON 문자열로 변경

In [0]:
from pyspark.sql.functions import to_json

df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")))

Out[9]: DataFrame[to_json(myStruct): string]

* to_json 함수 : JSON 데이터소스와 동일한 형태의 딕셔너리(맵)를 파라미터로 사용 가능. 
* from_json 함수 : JSON 문자열을 다시 객체로 변환. 단, 파라미터로 반드시 스키마 지정. (맵 데이터 타입의 옵션을 인수로 지정 가능)

In [0]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType((
  StructField("InvoiceNo", StringType(), True),
  StructField("Description", StringType(), True)))

df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("newJSON"))\
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)

+--------------------+--------------------+
 from_json(newJSON)| newJSON|
+--------------------+--------------------+
{536365, WHITE HA...|{"InvoiceNo":"536...|
{536365, WHITE ME...|{"InvoiceNo":"536...|
+--------------------+--------------------+
only showing top 2 rows

## 6.11 사용자 정의 함수

UDF
* 파이썬이나 스칼라 그리고 외부 라이브러리를 사용해 사용자가 원하는 형태로 트랜스포메이션을 만들 수 있게 함. 
* 하나 이상의 컬럼을 입력으로 받고, 반환 가능.
* 레코드별로 데이터를 처리하는 함수이기 때문에 독특한 포맷이나 도메인에 특화된 언어를 사용하지 않음.
* 특정 SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록.

ex) 숫자를 입력받아 세제곱 연산을 하는 power3 함수 생성 - 사용할 UDF를 만들기 위해 함수 필요

In [0]:
udfExampleDF = spark.range(5).toDF("num")

def power3(double_value):
  return double_value ** 3

power3(2.0)

Out[11]: 8.0

모든 워커 노드에서 생성된 함수를 사용할 수 있도록 스파크에 등록하는 과정 필요.

< 파이썬 UDF 처리 과정 >
1. 함수 직렬화 후 워커에 전달
2. 스파크에서 파이썬 프로세스 실행 후 데이터 전송
3. 파이썬에서 처리 결과 반환

ex) DataFrame에서 사용할 수 있도록 함수 등록

In [0]:
from pyspark.sql.functions import udf

power3udf = udf(power3)

사용자 정의 함수 등록 후 DataFrame에서 사용 가능

In [0]:
from pyspark.sql.functions import col

udfExampleDF.select(power3udf(col("num"))).show(2)

+-----------+
power3(num)|
+-----------+
 0|
 1|
+-----------+
only showing top 2 rows

사용자 정의 함수를 DataFrame에서만 사용이 가능하고 문자열 표현식에서는 사용 불가능.

하지만 사용자 정의 함수를 스파크 SQL 함수로 등록하면 모든 프로그래밍 언어와 SQL에서 사용자 정의 함수 사용 가능.

스파크는 파이썬의 데이터 타입과 다른 자체 데이터 타입을 사용하므로 함수를 정의할 때 반환 타입을 지정하는 것이 좋음. 
(만약 함수에서 반환될 실제 데이터 타입과 일치하지 않는 데이터 타입 지정시 오류가 아닌 null 값 반환)

ex) 함수의 반환 데이터 타입을 DoubleType으로 변경하면?

In [0]:
from pyspark.sql.types import IntegerType, DoubleType

spark.udf.register("power3py", power3, DoubleType())

udfExampleDF.selectExpr("power3py(num)").show(2)

+-------------+
power3py(num)|
+-------------+
 null|
 null|
+-------------+
only showing top 2 rows

null 값을 반환하는 이유 : range 메서드가 Integer 데이터 타입의 데이터를 만들기 때문. (파이썬에서 Integer 데이터 타입을 사용해 연산했다면 Float 데이터 타입(스파크의 Double 데이터 타입)으로 변환할 수 없음)

따라서 파이썬 함수가 Integer 데이터 타입 대신 Float 데이터 타입을 반환하도록 수정하면 null 값을 반환하지 않게 된다.

## 6.12 Hive UDF

하이브 문법을 이용해 만든 UDF/UDAF도 사용이 가능. 

단, SparkSession을 생성할 때 SparkSession.builder().enableHiveSupport()를 명시해 하이브 지원 기능 활성화.

하이브 지원 기능이 활성화되면 SQL로 UPF 등록 가능.